In [10]:
# 1. import libraries
import requests # to download html code
from bs4 import BeautifulSoup # to navigate through the html code
import pandas as pd
import numpy as np
import re

import config
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

hot100_df = pd.read_csv("artistsong.csv")
clustered_df = pd.read_csv("clustered.csv")

In [11]:
hot100_df

,artist,song
0,miley cyrus,flowers
1,sza,kill bill
2,morgan wallen,last night
3,sam smith & kim petras,unholy
4,"metro boomin, the weeknd & 21 savage",creepin'
...,...,...
95,yandel & feid,yandel 150
96,mac demarco,heart to heart
97,jordan davis,next thing you know
98,p!nk,never gonna not dance again


In [12]:
import pyinputplus as pyip   #to insert predefined user input functions
import random

user_song = str(input("Choose your song: "))    
rec_song = hot100_df["song"].sample().to_string(index=False)     #randomly reccomend another song with sample, .to_string avoit to put the index too
rec_artist = hot100_df.loc[hot100_df["song"] == rec_song, "artist"].to_string(index=False)   #associate the artist to the random reccomended song

x_artist = hot100_df.loc[hot100_df['song'].map(str.lower)  == user_song.lower(), 'artist'] # artist corresponding to song x
artist_question = pyip.inputYesNo('Did you mean ' + user_song.lower().title() + ' by ' + x_artist.to_string(index=False).title() + '?')  #this asks if the user mean an identical song but not in the list, if neither Y or N are given, it asks again

if artist_question == "yes":
    if user_song.lower() in hot100_df.values and user_song.lower() != rec_song:         #.lower() exclude typos from capital/lower letter "and user_song.lower() != rec_song" exclude to randomize including the same song
        print(f"The song you have chosen is " + user_song.title() + ". Why don't you try to listen to " + rec_song.title() + " by " + rec_artist.title())     #.title() gives results with capital first letter
    else:
        print("Your song is not in Billboard 100 Hot")
else:
    print("Your song is not in Billboard 100 Hot")

Choose your song: flowers
Did you mean Flowers by Miley Cyrus?y
The song you have chosen is Flowers. Why don't you try to listen to Players by Coi Leray


In [33]:
#User function again
x = str(input("Choose your song: "))
results = sp.search(q=x, limit=3,market="GB")
results

Choose your song: Hotel California


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Hotel+California&type=track&market=GB&offset=0&limit=3',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0ECwFtbIWEVNwjlrfc6xoL'},
       'href': 'https://api.spotify.com/v1/artists/0ECwFtbIWEVNwjlrfc6xoL',
       'id': '0ECwFtbIWEVNwjlrfc6xoL',
       'name': 'Eagles',
       'type': 'artist',
       'uri': 'spotify:artist:0ECwFtbIWEVNwjlrfc6xoL'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/2widuo17g5CEC66IbzveRu'},
     'href': 'https://api.spotify.com/v1/albums/2widuo17g5CEC66IbzveRu',
     'id': '2widuo17g5CEC66IbzveRu',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b2734637341b9f507521afa9a778',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e024637341b9f507521afa9a778',
       'width': 300},
      {'height': 64,
       'url': 'https://

In [34]:
#From the song to the track id
track_id=track_id=results["tracks"]["items"][0]["id"]
track_id

'40riOy7x9W7GXjyGp4pjAv'

In [35]:
#Just try the display of the song
from IPython.display import IFrame

#track_id = "1rfORa9iYmocEsnnZGMVC4"
#track_id= 'spotify:track:3hgl7EQwTutSm6PESsB7gZ'
IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [36]:
#audio feature of the chosen song
x_af = sp.audio_features(track_id)
x_af

[{'danceability': 0.579,
  'energy': 0.508,
  'key': 2,
  'loudness': -9.484,
  'mode': 1,
  'speechiness': 0.027,
  'acousticness': 0.00574,
  'instrumentalness': 0.000494,
  'liveness': 0.0575,
  'valence': 0.609,
  'tempo': 147.125,
  'type': 'audio_features',
  'id': '40riOy7x9W7GXjyGp4pjAv',
  'uri': 'spotify:track:40riOy7x9W7GXjyGp4pjAv',
  'track_href': 'https://api.spotify.com/v1/tracks/40riOy7x9W7GXjyGp4pjAv',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/40riOy7x9W7GXjyGp4pjAv',
  'duration_ms': 391376,
  'time_signature': 4}]